# Define Imports

In [ ]:
import pandas as pd
from transformers import pipeline
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Class Declarations

## Sentiment Analyzer Class

In [ ]:
class SentimentAnalyzer:
    """
    A class to perform sentiment analysis using a Hugging Face model.
    """
    def __init__(self, model_name="distilbert-base-uncased-finetuned-sst-2-english"):
        """
        Initializes the sentiment analysis pipeline.
        The challenge recommends this model.
        """
        print(f"Loading sentiment model: {model_name}...")
        # Using device=0 will use GPU if available, -1 for CPU
        self.sentiment_pipeline = pipeline("sentiment-analysis", model=model_name, device=-1)
        print("Model loaded successfully.")

    def analyze(self, df, text_column='review'):
        """
        Applies sentiment analysis to a DataFrame column.
        Args:
            df (pd.DataFrame): The input DataFrame.
            text_column (str): The column containing text to analyze.
        Returns:
            pd.DataFrame: The DataFrame with added 'sentiment_label' and 'sentiment_score' columns.
        """
        print("Applying sentiment analysis...")
        # The pipeline returns a list of dictionaries like {'label': 'POSITIVE', 'score': 0.999}
        # We need to handle potential long texts by truncating them for the model
        sentiments = self.sentiment_pipeline(df[text_column].fillna('').tolist(), truncation=True)
        
        # Extract labels and scores
        df['sentiment_label'] = [s['label'] for s in sentiments]
        df['sentiment_score'] = [s['score'] for s in sentiments]
        print("Sentiment analysis complete.")
        return df

## ThematicAnalyzer

In [ ]:
class ThematicAnalyzer:
    """
    A class to perform thematic analysis using NLP techniques.
    """
    def __init__(self, stop_words='english'):
        self.vectorizer = TfidfVectorizer(max_features=100, ngram_range=(1, 3), stop_words=stop_words)

    def preprocess_text(self, text):
        """
        Lemmatizes text and removes stopwords and punctuation.
        """
        doc = nlp(text.lower())
        lemmas = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct and token.is_alpha]
        return " ".join(lemmas)
    
    def extract_keywords(self, df, text_column='review'):
        """
        Extracts top keywords for each bank using TF-IDF.
        """
        print("Extracting keywords using TF-IDF...")
        bank_keywords = {}
        for bank in df['bank'].unique():
            print(f"  - Processing for {bank}")
            bank_df = df[df['bank'] == bank]
            processed_reviews = bank_df[text_column].apply(self.preprocess_text)
            
            tfidf_matrix = self.vectorizer.fit_transform(processed_reviews)
            feature_names = self.vectorizer.get_feature_names_out()
            bank_keywords[bank] = feature_names
        print("Keyword extraction complete.")
        return bank_keywords

    def assign_themes(self, df, text_column='review'):
        """
        Assigns predefined themes based on keyword matching.
        This is a rule-based approach as suggested by the challenge.
        """
        print("Assigning themes...")
        # Define keywords for each theme
        theme_map = {
            'Account & Login': ['login', 'account', 'password', 'register', 'signin', 'otp'],
            'Transactions & Transfers': ['transfer', 'transaction', 'payment', 'send', 'money', 'slow', 'fast', 'fee'],
            'UI & Experience': ['ui', 'interface', 'design', 'easy', 'simple', 'update', 'dark mode'],
            'Bugs & Performance': ['bug', 'crash', 'error', 'slow', 'performance', 'stuck', 'fix', 'issue'],
            'Features & Services': ['feature', 'service', 'loan', 'statement', 'fingerprint', 'biometric']
        }

        def find_theme(review_text):
            review_text = review_text.lower()
            found_themes = []
            for theme, keywords in theme_map.items():
                if any(keyword in review_text for keyword in keywords):
                    found_themes.append(theme)
            return ", ".join(found_themes) if found_themes else 'General Feedback'

        df['themes'] = df[text_column].apply(find_theme)
        print("Theme assignment complete.")
        return df

## Load cleaned data from task-1

In [ ]:
df_cleaned = pd.read_csv('../data/cleaned_play_store_reviews.csv')

## Perform Sentiment Analysis

In [ ]:
sentiment_analyzer = SentimentAnalyzer()
df_sentiment = sentiment_analyzer.analyze(df_cleaned.copy())

## Perform Thematic Analysis

In [ ]:
thematic_analyzer = ThematicAnalyzer()
df_final = thematic_analyzer.assign_themes(df_sentiment.copy())

## Display Top Keywords Per Bank

In [ ]:
keywords_per_bank = thematic_analyzer.extract_keywords(df_final.copy())
for bank, keywords in keywords_per_bank.items():
    print(f"\nTop keywords for {bank}: {', '.join(keywords[:10])}")

## Finalize and Save

In [ ]:
df_final.reset_index(inplace=True)
df_final.rename(columns={
    'index': 'review_id',
    'review': 'review_text',
    'themes': 'identified_theme(s)'
}, inplace=True)

# Select and reorder columns
output_columns = ['review_id', 'review_text', 'sentiment_label', 'sentiment_score', 'identified_theme(s)', 'rating', 'date', 'bank']
df_output = df_final[output_columns]

print("\nFinal DataFrame Head:")
print(df_output.head())

# Save the results to a new CSV file 
output_path = '../data/analyzed_reviews.csv'
df_output.to_csv(output_path, index=False, encoding='utf-8')
print(f"\nAnalyzed data with sentiment and themes saved to {output_path}")